In [1]:
import pandas as pd
import numpy as np
import re

In [11]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [15]:
# read all the csvv files
df = pd.read_csv("findmyguitars.csv")
neck_df = pd.read_csv("neck_df.csv")
bridge_df = pd.read_csv("bridge_df.csv")
body_shape_df = pd.read_csv("body_shape_df.csv")

In [13]:
df.shape,neck_df.shape,bridge_df.shape,body_shape_df.shape

((1663, 90), (1663, 2), (1349, 2), (1663, 2))

In [14]:
# drop the row that has no price information
df = df[~pd.isnull(df.Price)]
df.shape

(1604, 90)

In [6]:
# join the dataframes by url values
df = pd.merge(df, neck_df, how="left", on='Link')
df = pd.merge(df, bridge_df,how='left', on='Link')
df = pd.merge(df, body_shape_df,how='left', on='Link')

df.head()

,Link,Model,Specs,Matched,Brand,Year,Configuration,Strings,Made in,Series,Colors,Left-Handed Version,Top,Type,Body Material,Bridge,Neck Joint,Tuners,Fretboard,Neck Material,Decoration,Scale Size,Shape,Thickness,Frets,Fretboard Radius,Nut,Nut Width,Switch,Knobs,Pickup Mods,Volume Controls,Tone Controls,Bridge Pickup,Neck Pickup,Middle Pickup,Price,Tremolo,Locking Tuners,Retainer Bar,Stainless Steel Frets,Made in United States,Expensive Wood,Locking Nut,Top Pickup Brand,Neck-Through Build,Push Knob or Extra Switch Option,Weight Relief,Stays in Tune (Evertune),21:1 Tuner Ratio,Compound Radius Fretboard,Strap Lock,Luminescent Inlay,Body Material 2,Neck Material 2,Fretboard Material 2,Sustainer Pickups,High-Quality-Standards Country,Coil Split Pickups,Made in Japan,"Coil Split, Killswitch Pickups","Coil Split, Multi-Voicing Pickups","Multi-Voicing, Sustainer Pickups",Coil Tap Pickups,Ivory Tusq Nut,Bone Nut,Multi-Voicing Pickups,High-Quality Nut,Black Tusq XL Nut,Series Split Pickups,Parallel Split Pickups,S-1 Switch Pickups,Killswitch Pickups,"Coil Split, Capacitance Selector Pickups","Coil Tap, Phase Out Pickups",Boneite Nut,NuBone Nut,Synthetic Bone Nut,"Killswitch, Sustainer Pickups",Boost Pickups,"Multi-Voicing, Gain Reduction Pickups","Coil Split, Boost Pickups",Phase Out Pickups,"Killswitch, Phase Out Pickups",High-Pass Filter Pickups,Half-Bone/Half-Brass Nut,"Coil Tap, Boost Pickups","Coil Split, Phase Out Pickups",Varitone Pickups,Tri-Sound Pickups,Neck Profile,Bridge Type,Body Shape
0,https://findmyguitar.com/reviews/Schecter-Bans...,Schecter Banshee Mach-6 FR S,From Schecter's 2020 Banshee Mach series\nMade...,False,Schecter,2020.0,HH,6.0,South Korea,Banshee Mach,"Black Burst, Natural Burst",Yes,Flamed Maple,Solid Body,Swamp Ash,Floyd Rose 1500 Series,Neck-Through,Grover Rotomatic 18:1,Ebony,Roasted Maple/Purpleheart Multi-ply with Carbo...,Offset/Reverse Roman Numerals,"25.5""",Ultra Thin U,1st Fret: 0.748'' (19mm) - 12th Fret: 0.787'' ...,24 XL Jumbo Yes,"12"" to 16""",Locking,41.3mm (1.625''),3 Way,Dome,Sustainer,1.0,1.0,Lundgren M6 (Humbucker / Passive),Sustainiac (Humbucker / Passive),NaN,1357.600,True,False,True,True,NaN,True,True,True,True,NaN,False,False,False,True,False,True,Ash,Roasted Maple,Ebony,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,Floyd Rose,Super Strat
1,https://findmyguitar.com/reviews/ESP-E-II-Hori...,ESP E-II Horizon FR-II,From ESP E-II's 2021 Horizon series\nMade in J...,True,ESP E-II,2021.0,HH,6.0,Japan,Horizon,"Tiger Eye Sunburst, See Thru Black Cherry Sunb...",Yes,Quilted Maple,Solid Body,Mahogany,Floyd Rose Original,Neck-Through,Gotoh Locking,Ebony,3pc Maple,Offset Blocks,"25.5""",Thin U,1st Fret: 0.787'' (20mm) - 12th Fret: 0.866'' ...,24 XL Jumbo,"12""",Locking,42mm (1.654''),3 Way,Dome,Coil Split,1.0,1.0,EMG 57TW (Humbucker / Active),EMG 66TW (Humbucker / Active),NaN,2308.628,True,True,True,False,NaN,True,True,True,True,NaN,False,False,False,False,True,False,Mahogany,Maple,Ebony,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,Floyd Rose,Super Strat
2,https://findmyguitar.com/reviews/Jackson-USA-S...,Jackson USA Signature Chris Broderick Soloist 6,From Jackson's 2016 Artist Signature series\nC...,False,Jackson,2016.0,HH,6.0,United States,Artist Signature,"White, Black, Red",No,1/8 Plain Maple,Solid Body,Mahogany,Floyd Rose Pro (Recessed),Neck-Through,Planet Waves Auto-Trim,Ebony,Quartersawn Maple,NaN,"25.5""",Jackson Standard,1st Fret: 0.79'' (20.1mm) - 12th Fret: 0.85'' ...,24 Jumbo,"12""",Locking,42.9mm (1.688''),3 Way,Dome,"Coil Split, Killswitch",1.0,1.0,Direct Mount Chris Broderick Designed Custom D...,Direct Mount Chris Broderick Designed Custom D...,NaN,3346.450,True,True,False,False,True,True,True,False,True,NaN,False,False,False,False,True,False,Mahogany,Maple,Ebony,NaN,NaN,NaN,NaN,True,NaN,NaN,N

In [10]:
df.isnull().sum()

Link                                           0
Model                                          0
Specs                                          2
Matched                                        2
Brand                                          0
Year                                           0
Configuration                                  0
Strings                                        0
Made in                                        0
Series                                         0
Colors                                         0
Left-Handed Version                            0
Top                                          820
Type                                           0
Body Material                                  0
Bridge                                         0
Neck Joint                                     0
Tuners                                         0
Fretboard                                      0
Neck Material                                  0
Decoration          

In [32]:
df['Bridge Type'].value_counts()

Fixed             462
Tremolo           316
Floyd Rose        312
Bigsby Tremolo     79
Evertune           76
Edge               17
Edge Zero II       16
Kahler             13
Lo-Pro Edge         8
Edge Zero           2
Name: Bridge Type, dtype: int64

In [33]:
def fill_bridge_type(x):
    if pd.isnull(x['Bridge Type']):
        if "Fixed" in x['Bridge']:
            return "Fixed"
        else:
            return None
    else:
        return x['Bridge Type']
    
    
df['Bridge Type'] = df.apply(fill_bridge_type,axis=1)

In [34]:
bridges_and_types = df.groupby(['Bridge','Bridge Type'],dropna=False)['Bridge Type'].size().unstack()
bridges_and_types[bridges_and_types.iloc[:,-1] > 0]

Bridge Type,Bigsby Tremolo,Edge,Edge Zero,Edge Zero II,Evertune,Fixed,Floyd Rose,Kahler,Lo-Pro Edge,Tremolo,NaN
Bridge,,,,,,,,,,,
3-Saddle Custom Cut-Off Vintage-Style Tele Bridge with Compensated Brass Saddles,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
3-Saddle Vintage-Style Strings Through Body Tele with Compensated Brass Barrel Saddles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3-Saddle Vintage-Style Strings-Through-Body Tele with Brass Barrel Saddles,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0
3-Saddle Vintage-Style Strings-Through-Body Tele with Chrome Barrel Saddles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
3-Saddle Vintage-Style Tele with Slotted Steel Saddles,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0
6-Saddle American Vintage Strings-Through-Body Strat Hardtail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6-Saddle Hardtail,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
6-Saddle Hardtail with Cast Saddles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6-Saddle Standard Strings-Through-Body Tele with Block Saddles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [8]:
df.Tuners.value_counts()

Grover                                                                          101
Jackson Sealed Die-Cast                                                          98
Ibanez                                                                           76
Gotoh Locking                                                                    75
Schecter Locking                                                                 75
LTD Locking                                                                      72
Solar 18:1 Locking Type                                                          63
Vintage-Style                                                                    62
Schecter                                                                         50
Reverend Pin-Lock                                                                50
LTD                                                                              42
Gotoh MG-T locking machine heads                                            

In [35]:
df['Bridge Type'].fillna(value='Fixed',inplace=True)

In [36]:
df[df['Expensive Wood'] == False][['Body Material 2','Neck Material 2','Fretboard Material 2']]

,Body Material 2,Neck Material 2,Fretboard Material 2
7,Basswood,Maple,Maple
76,Basswood,Maple,Maple
79,Alder,Roasted Maple,Roasted Maple
90,Alder,Roasted Maple,Roasted Maple
91,Alder,Roasted Maple,Roasted Maple
92,Alder,Roasted Maple,Roasted Maple
93,Alder,Roasted Maple,Roasted Maple
94,Alder,Roasted Maple,Roasted Maple
95,Alder,Roasted Maple,Roasted Maple
97,Alder,Roasted Maple,Roasted Maple


In [37]:
cols = ['Brand','Year','Configuration','Strings','Made in','Type','Body Shape','Body Material 2','Neck Profile','Neck Joint','Fretboard Material 2','Neck Material 2','Expensive Wood','Bridge Type','Bridge','Scale Size','Shape','Frets','Fretboard Radius','Stainless Steel Frets','Nut','Nut Width','Switch','Knobs','Volume Controls','Tone Controls','Bridge Pickup','Neck Pickup','Middle Pickup','Top Pickup Brand','Locking Tuners','Tremolo','Stays in Tune (Evertune)','Luminescent Inlay','Compound Radius Fretboard','Price']

In [38]:
df = df[cols]

In [39]:
df = df.rename(columns={'Body Material 2' : 'Body Material', 'Neck Material 2' : 'Neck Material', 'Fretboard Material 2' : 'Fretboard Material'})

In [40]:
df['Brand'].value_counts()

Schecter         194
Ibanez           170
ESP LTD          154
Jackson          146
Fender           112
Harley Benton     98
Solar             90
Gretsch           86
Dean              71
Charvel           58
PRS               58
Reverend          50
Gibson            43
Epiphone          40
Fender Squier     40
ESP               35
ESP E-II          33
Sterling          32
Music Man         30
Yamaha            22
Kramer            18
Washburn          15
Sire               7
Cort               2
Name: Brand, dtype: int64

In [41]:
df['Body Material'].value_counts()

Mahogany         635
Alder            304
Basswood         145
Ash              134
Poplar           107
Maple             86
Limba             50
Nyatoh            45
Okoume            30
Meranti           19
Flame Maple       13
Spruce             6
Sungkai            6
Sassafras          4
Sapele             4
Terentang          3
Pine               3
Agathis            2
Figured Maple      2
Jabon              2
Nato               2
Padauk             1
Paulownia          1
Name: Body Material, dtype: int64

In [42]:
df['Body Material'] = df['Body Material'].apply(lambda x : 'Maple' if 'Maple' in x else x) 

In [43]:
df['Neck Material'].value_counts()

Maple             1029
Mahogany           354
Roasted Maple      141
Limba               19
Figured Maple       18
Nato                11
Panga Panga          8
Flame Maple          6
Wenge                6
Birdseye Maple       5
Nyatoh               5
Padauk               1
Okoume               1
Name: Neck Material, dtype: int64

In [44]:
df['Neck Material'] = df['Neck Material'].apply(lambda x : 'Maple' if 'Maple' in x else x) 

In [45]:
df['Fretboard Material'].value_counts()

Ebony             644
Rosewood          347
Maple             187
Laurel            107
Jatoba             75
Roasted Maple      71
Purpleheart        50
Pau Ferro          49
Figured Maple      11
Flame Maple        11
Roseacer           11
Wenge               9
Birdseye Maple      7
Blackwood           7
Walnut              6
Engineered          5
Granadillo          3
Richlite            2
Sungkai             1
Terentang           1
Name: Fretboard Material, dtype: int64

In [46]:
df['Fretboard Material'] = df['Fretboard Material'].apply(lambda x : 'Maple' if 'Maple' in x else x) 

In [51]:
df['Frets'].value_counts()

XL Jumbo             496
Jumbo                493
Medium Jumbo         427
Medium                93
Narrow Tall           47
Vintage               26
Vintage Tall          14
Jumbo Evo Gold         6
Medium Evo Gold        1
XL Jumbo Evo Gold      1
Name: Frets, dtype: int64

In [48]:
# yes indicates that frets are Stainless Steel Frets which we have the information
df['Frets'] = df['Frets'].apply(lambda x : x.replace(" Yes",'')) 

In [49]:
df['Number of Frets'] = df['Frets'].apply(lambda x : str(x.split()[0]))

In [50]:
# we can delete the frets numbers from the frets column
df['Frets'] = df['Frets'].apply(lambda x : re.sub('\d{2} ','',x))

In [123]:
df['Number of Frets'].value_counts()

24    787
22    723
21     91
20      2
27      1
Name: Number of Frets, dtype: int64

In [52]:
df['Nut Width'].value_counts()

42mm (1.654'')      464
43mm (1.693'')      416
42.9mm (1.688'')    287
48mm (1.89'')       115
41.3mm (1.625'')     88
41.9mm (1.65'')      39
42.8mm (1.685'')     38
47.6mm (1.875'')     27
54mm (2.126'')       25
41mm (1.614'')       21
42.1mm (1.656'')     17
44.5mm (1.75'')      13
55mm (2.165'')        9
49.5mm (1.95'')       7
42.7mm (1.68'')       6
47mm (1.85'')         5
40.5mm (1.594'')      4
41.7mm (1.643'')      4
45mm (1.77'')         3
63.5mm (2.5'')        2
40mm (1.575'')        2
42.5mm (1.675'')      2
40.1mm (1.58'')       1
40.6mm (1.6'')        1
42.4mm (1.67'')       1
62mm (2.44'')         1
43.4mm (1.71'')       1
46mm (1.811'')        1
50mm (1.969'')        1
mm ('')               1
57.2mm (2.252'')      1
52.4mm (2.062'')      1
Name: Nut Width, dtype: int64

In [53]:
# missing nut width guitar is Schecter Synyster Gates FR and it's nut width is 41.3mm
missing_nut_width_idx = df[df['Nut Width'] == "mm ('')"].index[0]
df.at[missing_nut_width_idx,'Nut Width'] = "41.3mm (1.625'')"

In [140]:
df['Nut Width'] = df['Nut Width'].apply(lambda x : float(x.split("mm")[0]))
df = df.rename(columns = {'Nut Width' : 'Nut Width (mm)'})
df["Nut Width (mm)"].value_counts()

42.0    463
43.0    416
42.9    286
48.0    115
41.3     89
41.9     39
42.8     38
47.6     27
54.0     25
41.0     21
42.1     17
44.5     13
55.0      9
49.5      7
42.7      6
47.0      5
41.7      4
40.5      4
45.0      3
63.5      2
40.0      2
42.5      2
43.4      1
62.0      1
46.0      1
40.6      1
50.0      1
40.1      1
57.2      1
42.4      1
Name: Nut Width (mm), dtype: int64

In [131]:
pickup_brands = ['Artec','Bare Knuckle','Bill Lawrence','Charvel Custom','DiMarzio','DMT','Duncan Solar','EMG','ESP','Fender'
                 ,'Fishman','Gibson','Gretsch','Lundgren','Mojotone','Music Man','PRS','Reverend','Roswell','Schecter','Jackson',
                 'Seymour Duncan','Tesla','Yamaha','Blackfire']

pickup_brand_map = {'Talnico Bridge' : 'Reverend','Reeves Gabrels Signature Bridge' : 'Railhammer', 'VP5b' : None,'VH5b':None,
                   'Quantum' : 'Ibanez', 'HA5' : 'Reverend', '9A5' : 'Reverend', 'Hyper Vintage' : 'Railhammer', 'Classic Elite' : 'Ibanez',
                   'Duncan Designed' : 'Seymour Duncan', 'Chisel Bridge' : 'Railhammer', 'Sterling' : 'Music Man',
                   'HB Humbucker' : 'Harley Benton', 'Infinity R' : 'Ibanez', 'Alnico Classic PRO' : 'Epiphone', 'ProBucker 3' : 'Epiphone',
                   'Super 58' : 'Ibanez', 'HB Single Coil' : 'Harley Benton', 'Epiphone' : 'Epiphone', 'IBZ' : 'Ibanez', 'Squier' : 'Fender Squier',
                   'Hi-Gain' : 'Harley Benton', 'Kramer' : 'Kramer', 'V8' : 'Ibanze'
                   } 

def get_pickup_brand(x):
    if pd.isnull(x):
        return "No Pickup"
    
    for b in pickup_brands:
        if b.lower() in x.lower():
            return b
    return None

def map_pickup_brand(x):

    if pd.isnull(x.iloc[0]):
        return "No Pickup"
    
    if pd.isnull(x.iloc[1]): 
        for pu, brand in pickup_brand_map.items():
            #print(x,pu,brand)
            if pu in x.iloc[0]:
                return brand
        return None
    else:
        return x.iloc[1]

In [139]:
df['Bridge Pickup Brand'] = df['Bridge Pickup'].apply(get_pickup_brand)
df['Bridge Pickup Brand'] = df[['Bridge Pickup','Bridge Pickup Brand']].apply(map_pickup_brand,axis=1)

df['Middle Pickup Brand'] = df['Middle Pickup'].apply(get_pickup_brand)

df['Neck Pickup Brand'] = df['Neck Pickup'].apply(get_pickup_brand)
#df['Neck Pickup Brand'] = df[['Neck Pickup','Neck Pickup Brand']].apply(map_pickup_brand,axis=1)

In [140]:
# 
df[pd.isnull(df['Bridge Pickup Brand'])][['Bridge Pickup']].value_counts()

Bridge Pickup                                                                   
Alnico Humbucker (Humbucker / Passive)                                              6
Standard Humbucking (Humbucker / Passive)                                           5
Ceramic Humbucker (Humbucker / Passive)                                             4
Washburn Humbucker (Humbucker / Passive)                                            2
Sire Super-ST Humbucker (Humbucker / Passive)                                       2
Retroblast Bridge (Humbucker / Passive)                                             2
Cleancut Bridge (Humbucker / Passive)                                               2
Billy Corgan Bridge (Humbucker / Passive)                                           2
Atomic Humbucking (Humbucker / Passive)                                             2
Player Plus Noiseless Tele (Single Coil / Passive)                                  2
Standard Single-Coil Strat (Single Coil / Passive)         

In [137]:
df['Neck Pickup Brand'].value_counts(dropna=False)

None              298
Seymour Duncan    224
EMG               129
Fender            112
Fishman           105
DiMarzio           93
Schecter           91
Gretsch            85
Duncan Solar       77
No Pickup          75
PRS                55
Jackson            45
Roswell            42
Gibson             38
DMT                33
ESP                33
Music Man          19
Bare Knuckle       14
Lundgren           11
Artec               6
Blackfire           4
Charvel Custom      4
Tesla               4
Mojotone            3
Yamaha              1
Name: Neck Pickup Brand, dtype: int64

In [130]:
df[df['Bridge Pickup'] == 'V8 (Humbucker / Passive)'].iloc[0,0]

'https://findmyguitar.com/reviews/Ibanez-RG550-Genesis-Collection-review.php'

In [54]:
def get_pickup_power(x):
    result = "Active" if "Active" in x['Bridge Pickup'] else "Passive"
    
    if not pd.isnull(x['Middle Pickup']):
        if "Active" in x['Middle Pickup']:
            result += "-Active"
        else:
            result += "-Passive"
            
    if not pd.isnull(x['Neck Pickup']):
        if "Active" in x['Neck Pickup']:
            result += "-Active"
        else:
            result += "-Passive"
            
    return result

In [55]:
df['Pickups Power'] = df.apply(get_pickup_power,axis=1)

In [28]:
df['Pickups Power'].value_counts()

Passive-Passive            1028
Active-Active               254
Passive-Passive-Passive     227
Passive                      48
Active                       22
Active-Passive               15
Active-Active-Active          5
Passive-Passive-Active        3
Active-Passive-Passive        1
Passive-Active-Passive        1
Name: Pickups Power, dtype: int64

In [29]:
df['Configuration'].value_counts()

HH           1138
HSS           112
SS             73
SSS            66
H              60
HSH            38
P90P90         30
HS             25
SH             16
HHH            15
P90            10
HP90            7
HHX             5
P90P90P90       3
SP90            2
SSH             1
HHS             1
HP90P90         1
XXH             1
Name: Configuration, dtype: int64

In [51]:
df['Stainless Steel Frets'].value_counts()

False    1302
True      299
Name: Stainless Steel Frets, dtype: int64

In [52]:
df['Top Pickup Brand'].value_counts()

True     1207
False     394
Name: Top Pickup Brand, dtype: int64

In [53]:
df['Locking Tuners'].value_counts()

False    971
True     630
Name: Locking Tuners, dtype: int64

In [54]:
df['Stays in Tune (Evertune)'].value_counts()

False    1525
True       76
Name: Stays in Tune (Evertune), dtype: int64

In [55]:
df.head()

,Brand,Year,Configuration,Strings,Made in,Type,Body Material,Neck Joint,Fretboard Material,Neck Material,Expensive Wood,Bridge,Scale Size,Shape,Frets,Fretboard Radius,Stainless Steel Frets,Nut,Nut Width,Switch,Knobs,Volume Controls,Tone Controls,Bridge Pickup,Neck Pickup,Middle Pickup,Top Pickup Brand,Locking Tuners,Tremolo,Stays in Tune (Evertune),Luminescent Inlay,Compound Radius Fretboard,Price,Number of Frets,Pickups Power
0,Schecter,2020,HH,6,South Korea,Solid Body,Ash,Neck-Through,Ebony,Maple,True,Floyd Rose 1500 Series,"25.5""",Ultra Thin U,24 XL Jumbo Yes,"12"" to 16""",True,Locking,41.3mm (1.625''),3 Way,Dome,1,1,Lundgren M6 (Humbucker / Passive),Sustainiac (Humbucker / Passive),NaN,True,False,True,False,True,True,1357.600,24,Passive-Passive
1,ESP E-II,2021,HH,6,Japan,Solid Body,Mahogany,Neck-Through,Ebony,Maple,True,Floyd Rose Original,"25.5""",Thin U,24 XL Jumbo,"12""",False,Locking,42mm (1.654''),3 Way,Dome,1,1,EMG 57TW (Humbucker / Active),EMG 66TW (Humbucker / Active),NaN,True,True,True,False,False,False,2308.628,24,Active-Active
2,Jackson,2016,HH,6,United States,Solid Body,Mahogany,Neck-Through,Ebony,Maple,True,Floyd Rose Pro (Recessed),"25.5""",Jackson Standard,24 Jumbo,"12""",False,Locking,42.9mm (1.688''),3 Way,Dome,1,1,Direct Mount Chris Broderick Designed Custom D...,Direct Mount Chris Broderick Designed Custom D...,NaN,False,True,True,False,False,False,3346.450,24,Passive-Passive
3,Schecter,2021,HH,6,Indonesia,Solid Body,Ash,Neck-Through,Ebony,Maple,True,Floyd Rose 1500 Series,"25.5""",Ultra Thin C,24 XL Jumbo Yes,"12"" to 16""",True,Locking,41.3mm (1.625''),3 Way,Dome,1,1,Fishman Fluence Modern Ceramic (Humbucker / Ac...,Fishman Fluence Modern Alnico (Humbucker / Act...,NaN,True,False,True,False,True,True,1214.245,24,Active-Active
4,Schecter,2020,HH,7,South Korea,Solid Body,Ash,Neck-Through,Ebony,Maple,True,Floyd Rose 1500 Series,"26.5""",Ultra Thin U,24 XL Jumbo Yes,"12"" to 16""",True,Locking,47.6mm (1.875''),3 Way,Dome,1,1,Lundgren M7 (Humbucker / Passive),Sustainiac (Humbucker / Passive),NaN,True,False,True,False,True,True,1487.980,24,Passive-Passive


In [56]:
def outliers(df,feature):
    q1 = df[feature].quantile(0.25)
    q3 = df[feature].quantile(0.75)
    iqr = q3 - q1
    
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    
    idx = df.index[(df[feature] < lower) | (df[feature] > upper)]
    
    return idx

In [57]:
#deneme = pd.read_csv("findmyguitars.csv")
#deneme = deneme[deneme.Price < 2200]

In [58]:
idx_list = outliers(df,'Price')
idx_list

Int64Index([   2,    8,    9,   12,   13,   14,   15,   16,   17,   23,
            ...
            1009, 1010, 1012, 1013, 1015, 1019, 1023, 1065, 1110, 1124],
           dtype='int64', length=159)

In [59]:
removed_outliers = df.drop(idx_list)

In [60]:
removed_outliers.to_csv("deneme.csv",index=False)

In [61]:
df.to_csv("cleaned_data.csv",index=False)

In [61]:
df.Shape.value_counts()

Thin U                                             197
Thin C                                             121
Speed                                               99
Solar C Shape                                       90
Modern C                                            79
Charvel Speed                                       55
Ultra Thin C                                        50
Standard U                                          48
Medium Oval                                         47
Harley Benton C                                     43
Jackson Standard                                    37
Extra Thin U                                        35
Wizard III                                          32
Harley Benton Modern C                              31
V                                                   30
AZ Oval C                                           26
C                                                   26
Slim Taper                                          25
John Petru

In [60]:
d = pd.read_csv("findmyguitars.csv")
d[(d.Brand == 'Ibanez') & (d.Year == 2020) & (d['Body Material 2'] == 'Meranti') & (d['Fretboard Material 2'] == 'Jatoba') &
  (d.Strings == 6)]

,Link,Model,Specs,Matched,Brand,Year,Configuration,Strings,Made in,Series,Colors,Left-Handed Version,Top,Type,Body Material,Bridge,Neck Joint,Tuners,Fretboard,Neck Material,Decoration,Scale Size,Shape,Thickness,Frets,Fretboard Radius,Nut,Nut Width,Switch,Knobs,Pickup Mods,Volume Controls,Tone Controls,Bridge Pickup,Neck Pickup,Middle Pickup,Price,Tremolo,Locking Tuners,Retainer Bar,Stainless Steel Frets,Made in United States,Expensive Wood,Locking Nut,Top Pickup Brand,Neck-Through Build,Push Knob or Extra Switch Option,Weight Relief,Stays in Tune (Evertune),21:1 Tuner Ratio,Compound Radius Fretboard,Strap Lock,Luminescent Inlay,Body Material 2,Neck Material 2,Fretboard Material 2,Sustainer Pickups,High-Quality-Standards Country,Coil Split Pickups,Made in Japan,"Coil Split, Killswitch Pickups","Coil Split, Multi-Voicing Pickups","Multi-Voicing, Sustainer Pickups",Coil Tap Pickups,Ivory Tusq Nut,Bone Nut,Multi-Voicing Pickups,High-Quality Nut,Black Tusq XL Nut,Series Split Pickups,Parallel Split Pickups,S-1 Switch Pickups,Killswitch Pickups,"Coil Split, Capacitance Selector Pickups","Coil Tap, Phase Out Pickups",Boneite Nut,NuBone Nut,Synthetic Bone Nut,"Killswitch, Sustainer Pickups",Boost Pickups,"Multi-Voicing, Gain Reduction Pickups","Coil Split, Boost Pickups",Phase Out Pickups,"Killswitch, Phase Out Pickups",High-Pass Filter Pickups,Half-Bone/Half-Brass Nut,"Coil Tap, Boost Pickups","Coil Split, Phase Out Pickups",Varitone Pickups,Tri-Sound Pickups
1416,https://findmyguitar.com/reviews/Ibanez-RG421-...,Ibanez RG421,From Ibanez's 2020 RG series\nMade in Indonesi...,True,Ibanez,2020,HH,6,Indonesia,RG,Natural,No,Contour,Solid Body,Meranti,F106,Bolt-On,Ibanez,Jatoba,Maple,White Dot,"25.5""",Wizard III,1st Fret: 0.75'' (19.1mm) - 12th Fret: 0.83'' ...,24 Jumbo,"16""",Plastic,43mm (1.693''),5 Way,Dome,NaN,1,1,Quantum (Humbucker / Passive),Quantum (Humbucker / Passive),NaN,312.452500,False,False,NaN,False,NaN,False,NaN,False,False,False,False,False,False,False,False,False,Meranti,Maple,Jatoba,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1418,https://findmyguitar.com/reviews/Ibanez-RG421E...,Ibanez RG421EX,From Ibanez's 2020 RG series\nMade in Indonesi...,True,Ibanez,2020,HH,6,Indonesia,RG,Black,No,Contour,Solid Body,Meranti,F106,Bolt-On,Ibanez,Bound Jatoba,Maple,White Dot,"25.5""",Wizard III,1st Fret: 0.75'' (19.1mm) - 12th Fret: 0.83'' ...,24 Jumbo,"16""",Plastic,43mm (1.693''),5 Way,Dome,NaN,1,1,Quantum (Humbucker / Passive),Quantum (Humbucker / Passive),NaN,349.484000,False,False,NaN,False,NaN,False,NaN,False,False,False,False,False,False,False,False,False,Meranti,Maple,Jatoba,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1419,https://findmyguitar.com/reviews/Ibanez-RG421G...,Ibanez RG421G,From Ibanez's 2020 RG series\nMade in Indonesi...,False,Ibanez,2020,HH,6,Indonesia,RG,Blue,No,Contour,Solid Body,Meranti,F106,Bolt-On,Ibanez,Jatoba,Maple,White Dot,"25.5""",Wizard III,1st Fret: 0.75'' (19.1mm) - 12th Fret: 0.83'' ...,24 Jumbo,"16""",Plastic,43mm (1.693''),5 Way,Dome,NaN,1,1,Quantum (Humbucker / Passive),Quantum (Humbucker / Passive),NaN,342.727500,False,False,NaN,False,NaN,False,NaN,False,False,False,False,False,False,False,False,False,Meranti,Maple,Jatoba,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1421,https://findmyguitar.com/reviews/Ibanez-RG421P...,Ibanez RG421PB,From Ibanez's 2020 RG series\nMade in Indonesi...,True,Ibanez,2020,HH,6,Indonesia,RG,Blue Patterns,No,Contour Poplar Burl,Solid Body,Meranti,F106,Bolt-On,Ibanez,Jatoba,Maple,White Dot,"25.5""",Wizard III,1st Fret: 0.75'' (19.1mm) - 12th Fret: 0.83'' ...,24 Jumbo,"16""",Plastic,43mm (1.693''),5 Way,Dome,NaN,1,1,Quantum (Humbucker / Passive),Quantum (Humbucker / Passive),NaN,334.4960